In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import warnings # https://docs.python.org/3/library/warnings.html
warnings.filterwarnings('ignore')

In [ ]:
# Loading Data
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")
print(df_train.shape)
print(df_test.shape)

In [ ]:
# Label Encoding the Target Column
le = LabelEncoder()
df_train['target'] = le.fit_transform(df_train['target'])

mapping = dict(zip(le.classes_, range(0, len(le.classes_)+1)))
print(mapping)

In [ ]:
# Dividing Train and Test data
final_test = df_test.drop(["row_id"],axis = 1)
X = df_train.drop(["row_id","target"],axis = 1)
y = df_train['target']

In [ ]:
# Normalizing the data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
final_test = scaler.fit_transform(final_test)

In [ ]:
# Splitting data into 90,10 for train and validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.10, random_state = 42)
print('X_train', X_train.shape)
print('Y_train', y_train.shape)
print('X_test', X_test.shape)
print('Y_test', y_test.shape)

In [ ]:
# Sequential Model definition

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = Sequential()
model.add(Dense(143, activation='relu', kernel_initializer='normal', input_dim = 286))
model.add(Dense(70, activation='relu'))
model.add(Dense(10, activation='sigmoid',kernel_initializer='normal'))

# Compiling model for loss and optimized

opt = Adam(lr=0.001)
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = opt )

In [ ]:
# Model summary

model.summary()

In [ ]:
# Model Training

model.fit(X_train,y_train, epochs = 50, batch_size = 32,verbose = 1)

In [ ]:
# Prediction from model on validation set
result = model.predict(X_test)

#selecting the max output
pred = result.argmax(axis = 1)

In [ ]:
# Exploring the Prediction

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from matplotlib.pyplot import figure

print(f' Model Accuracy on Validation set : {accuracy_score(pred,y_test)}')


# Printing Confusion matrix for multiclass

cm = confusion_matrix(pred, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax,cmap=plt.cm.Blues)

In [ ]:
# Testing model on test data and submitting score

final_result = model.predict(final_test)
final_pred = final_result.argmax(axis = 1)

In [ ]:
final_prediction = le.inverse_transform(final_pred)

In [ ]:
# Creating Submission file to submit prediction
Submission = pd.DataFrame(data = {'row_id': df_test['row_id'], 'target': final_prediction})
Submission

In [ ]:
Submission.to_csv('Submission', index = False)
Submission.head()